### Basics

In [2]:
!pip install groq

In [3]:
from groq import Groq
import sqlite3
from termcolor import colored

In [23]:
api_key = "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"

In [22]:
import builtins

def wrap_text(text, max_length=150):
    lines = text.splitlines(keepends=True)
    wrapped_lines = []

    for line in lines:
        if line.strip() == "":
            wrapped_lines.append("")
            continue

        words = line.split()
        current_line = []

        for word in words:
            if len(' '.join(current_line + [word])) > max_length:
                wrapped_lines.append(' '.join(current_line))
                current_line = [word]
            else:
                current_line.append(word)

        if current_line:
            wrapped_lines.append(' '.join(current_line))

        if line.endswith("\n"):
            wrapped_lines[-1] += "\n"

    return '\n'.join(wrapped_lines)

def print(*args, max_length=150, **kwargs):
    text = ' '.join(str(arg) for arg in args)
    wrapped_text = wrap_text(text, max_length=max_length)
    builtins.print(wrapped_text, **kwargs)


In [ ]:
client = Groq(
    api_key=api_key
)

def generate_response(messages, model="llama-3.1-8b-instant", max_tokens=150, temperature=0.7):
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        max_tokens=max_tokens,
        temperature=temperature
    )

    response = chat_completion.choices[0].message
    return response


In [25]:
def add_user_message(messages, user_message):
    messages.append({"role": "user", "content": user_message})
    return messages

def add_ai_message(messages, ai_message):
    messages.append({"role": "assistant", "content": ai_message})
    return messages

def add_system_message(messages, system_message):
    messages.append({"role": "system", "content": system_message})
    return messages

In [34]:
# Set up the in-memory SQL database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create tables
cursor.execute("""
CREATE TABLE instructors (
    id INT PRIMARY KEY,
    name TEXT,
    email TEXT,
    department TEXT,
    office TEXT
)
""")

cursor.execute("""
CREATE TABLE courses (
    id INT PRIMARY KEY,
    name TEXT,
    description TEXT,
    instructor_id INT,
    quarter TEXT,
    FOREIGN KEY (instructor_id) REFERENCES instructors(id)
)
""")

cursor.execute("""
CREATE TABLE students (
    id INT PRIMARY KEY,
    name TEXT,
    email TEXT,
    advisor_id INT,
    FOREIGN KEY (advisor_id) REFERENCES instructors(id)
)
""")

cursor.execute("""
CREATE TABLE grades (
    id INT PRIMARY KEY,
    student_id INT,
    course_id INT,
    grade TEXT,
    FOREIGN KEY (student_id) REFERENCES students(id),
    FOREIGN KEY (course_id) REFERENCES courses(id)
)
""")

cursor.executemany("""
INSERT INTO instructors (id, name, email, department, office) VALUES (?, ?, ?, ?, ?)
""", [
    (1, "Tina Ostrander", "tina.ostrander@grc.edu", "Software Dev", "Auburn Center"),
    (2, "Susan Uland", "susan.uland@grc.edu", "Data and AI", "Auburn Center"),
    (3, "Josh Archer", "josh.archer@grc.edu", "Software Dev", "Office Auburn Center"),
    (4, "Auberon Lopez", "auberon.lopez@grc.edu", "Software Dev", "Office Auburn Center"),
    (5, "Keller Flint", "keller.flint@grc.edu", "Data and AI", "Zoom")
])

cursor.executemany("""
INSERT INTO courses (id, name, description, instructor_id, quarter) VALUES (?, ?, ?, ?, ?)
""", [
    (1, "Introduction to AI", "An introduction to artificial intelligence concepts and techniques.", 5, "Fall"),
    (2, "Machine Learning", "Covers supervised and unsupervised learning algorithms.", 2, "Winter"),
    (3, "Data Structures", "Introduction to data structures like lists, trees, and graphs.", 3, "Spring"),
    (4, "Operating Systems", "Covers the design and implementation of operating systems.", 4, "Fall"),
    (5, "Web Development", "Learn to build full-stack web applications.", 1, "Winter")
])

cursor.executemany("""
INSERT INTO students (id, name, email, advisor_id) VALUES (?, ?, ?, ?)
""", [
    (1, "John Doe", "john.doe@student.grc.edu", 1),
    (2, "Mei Chen", "jane.smith@student.grc.edu", 2),
    (3, "Valentine Wiggin", "emily.davis@student.grc.edu", 3),
    (4, "Michael Brown", "michael.brown@student.grc.edu", 4),
    (5, "Sarah Johnson", "sarah.johnson@student.grc.edu", 4),
    (6, "Robert Wilson", "robert.wilson@student.grc.edu", 1),
    (7, "Aisha Patel", "laura.white@student.grc.edu", 2),
    (8, "Daniel Garcia", "daniel.garcia@student.grc.edu", 3)
])

cursor.executemany("""
INSERT INTO grades (id, student_id, course_id, grade) VALUES (?, ?, ?, ?)
""", [
    (1, 1, 1, "C"),
    (2, 1, 3, "B"),
    (3, 2, 2, "A"),
    (4, 2, 4, "B"),
    (5, 3, 5, "A"),
    (6, 4, 1, "B"),
    (7, 5, 2, "A"),
    (8, 6, 3, "B"),
    (9, 7, 4, "B"),
    (10, 8, 5, "A")
])

conn.commit()

### RAG

In [ ]:
# Query the database
def retrieve_courses():
    cursor = conn.cursor()
    cursor.execute("""
        SELECT courses.name, description, instructors.name, quarter FROM courses 
        JOIN instructors ON courses.instructor_id = instructors.id;
    """)
    results = cursor.fetchall()
    return results, [description[0] for description in cursor.description]

# Advising Bot Workflow
def advising_bot(user_query):
    # Set up the database
    # Retrieve relevant courses
    courses, column_names = retrieve_courses()

    if not courses:
        context = "No courses found."
    else:
        context = "Here are the courses:\n"
        for course in courses:
            course_details = []
            for i in range(len(column_names)):
                course_details.append(f"{column_names[i]}: {course[i]}")

            context += ", ".join(course_details) + "\n"

    # Construct the LLM prompt
    prompt = f"""
    You are an academic advising assistant. Answer the user's query using the following course information:
    
    {context}
    
    User Query: {user_query}
    """

    # Generate a response
    messages = [{"role": "user", "content": prompt}]
    response = generate_response(messages, model="llama-3.1-8b-instant", max_tokens=300, temperature=0.2)

    return response

# Example Usage
user_query = input("Input your question") # "What courses are offered in Winter?" "What courses are offered by Susan Uland?"
response = advising_bot(user_query)
print("\nAdvising Bot Response:\n")
print(response)


Advising Bot Response:

Based on the provided course information, Susan Uland teaches the following class:


- Machine Learning (Winter quarter)


### Tools

Sight adjustment to our generate response - we will now want more than just the message content back and we're going to pass it in some tools it can use.

In [ ]:
client = Groq(
    api_key=api_key
)

def generate_response(messages, tools=[], model="llama-3.1-8b-instant", max_tokens=150, temperature=0.7):
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        max_tokens=max_tokens,
        temperature=temperature,
        tools=tools
    )

    response = chat_completion.choices[0].message
    return response

### Example Tool Call 
*Source: [Groq API Docs](https://console.groq.com/docs/tool-use)*

```json
{
  "model": "llama3-groq-70b-8192-tool-use-preview",
  "messages": [
    {
      "role": "system",
      "content": "You are a weather assistant. Use the get_weather function to retrieve weather information for a given location."
    },
    {
      "role": "user",
      "content": "What's the weather like in New York today?"
    }
  ],
  "tools": [
    {
      "type": "function",
      "function": {
        "name": "get_weather",
        "description": "Get the current weather for a location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g. San Francisco, CA"
            },
            "unit": {
              "type": "string",
              "enum": ["celsius", "fahrenheit"],
              "description": "The unit of temperature to use. Defaults to fahrenheit."
            }
          },
          "required": ["location"]
        }
      }
    }
  ],
  "tool_choice": "auto",
  "max_tokens": 4096
}'
```

### Example Tool Response
```json
"model": "llama3-groq-70b-8192-tool-use-preview",
"choices": [{
    "index": 0,
    "message": {
        "role": "assistant",
        "tool_calls": [{
            "id": "call_d5wg",
            "type": "function",
            "function": {
                "name": "get_weather",
                "arguments": "{\"location\": \"New York, NY\"}"
            }
        }]
    },
    "logprobs": null,
    "finish_reason": "tool_calls"
}],
```

In [58]:
# Predefined query functions
def get_courses_tool():
    cursor = conn.cursor()
    cursor.execute("""
        SELECT courses.name, description, instructors.name AS instructor, quarter 
        FROM courses 
        JOIN instructors ON courses.instructor_id = instructors.id;
    """)
    results = cursor.fetchall()
    return {"data": results, "columns": [description[0] for description in cursor.description]}

def get_students_tool():
    cursor = conn.cursor()
    cursor.execute("""
        SELECT students.name, students.email, instructors.name AS advisor 
        FROM students 
        JOIN instructors ON students.advisor_id = instructors.id;
    """)
    results = cursor.fetchall()
    return {"data": results, "columns": [description[0] for description in cursor.description]}

def get_instructors_tool():
    cursor = conn.cursor()
    cursor.execute("""
        SELECT name, email, department, office 
        FROM instructors;
    """)
    results = cursor.fetchall()
    return {"data": results, "columns": [description[0] for description in cursor.description]}

# Tool definitions
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_courses",
            "description": "Retrieve information about courses and who teaches them.",
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_students",
            "description": "Retrieve information about students and who their advisors are.",
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_instructors",
            "description": "Retrieve information about instructors.",
        }
    }
]

# Step 1: LLM selects tool
def generate_tool_call(user_query):
    # Construct the tool selection prompt
    messages = add_system_message([], "You are an academic advising assistant. Use the provided tools to retrieve relevant information based on the user's query.")
    messages = add_user_message(messages, user_query)

    # Call the LLM with tools defined
    response = generate_response(
        messages=messages,
        tools=tools,
        model="llama-3.1-8b-instant",
        max_tokens=50,
        temperature=0.2,
    )
    
    # Tool call extracted from LLM output
    return response.tool_calls[0].function.name

# Step 2: Execute selected tool
def execute_tool(tool_name):
    # Map tool functions
    tool_functions = {
        "get_courses": get_courses_tool,
        "get_students": get_students_tool,
        "get_instructors": get_instructors_tool
    }
    if tool_name in tool_functions:
        return tool_functions[tool_name]()
    else:
        raise ValueError(f"Unknown tool: {tool_name}")

# Step 3: LLM generates the final response using tool results
def generate_final_response(user_query, tool_name, tool_results):
    if not tool_results["data"]:
        context = f"No data was found using the {tool_name} tool."
    else:
        context = "Here is the retrieved information:\n"
        for row in tool_results["data"]:
            row_details = [f"{column}: {value}" for column, value in zip(tool_results["columns"], row)]
            context += ", ".join(row_details) + "\n"

    final_prompt = f"""
    You are an academic advising assistant. Answer the user's query using the following retrieved information:
    
    {context}
    
    User Query: {user_query}
    """

    messages = add_system_message([], final_prompt)

    final_response = generate_response(
        messages=messages,
        model="llama-3.1-8b-instant",
        max_tokens=300,
        temperature=0.2
    )
    return final_response

# Full workflow
def advising_bot_with_tool_calls(user_query):

    # Step 1: LLM decides which tool to call
    tool_name = generate_tool_call(user_query)

    # Print the LLM response
    print(colored("Tool Calls:", "cyan"), colored(tool_name, "blue"))

    # Step 2: Execute the tool
    tool_results = execute_tool(tool_name)

    print(colored(f"Tool Results for {tool_name}:", "green"), colored(tool_results, "blue"))

    # Step 3: Generate final response using tool results
    response = generate_final_response(user_query, tool_name, tool_results)

    return response

# Example Usage
user_query = input("Input your question: ")  # Examples: "What courses are offered in Winter?", "Who are the instructors?"
response = advising_bot_with_tool_calls(user_query)
print("\nAdvising Bot Response:\n")
print(response.content)


Tool Calls: get_courses
Tool Results for get_courses: {'data': [('Introduction to AI', 'An introduction to artificial intelligence concepts and techniques.',
'Keller Flint', 'Fall'), ('Machine Learning', 'Covers supervised and unsupervised learning algorithms.', 'Susan Uland', 'Winter'), ('Data Structures',
'Introduction to data structures like lists, trees, and graphs.', 'Josh Archer', 'Spring'), ('Operating Systems', 'Covers the design and
implementation of operating systems.', 'Auberon Lopez', 'Fall'), ('Web Development', 'Learn to build full-stack web applications.', 'Tina Ostrander',
'Winter')], 'columns': ['name', 'description', 'instructor', 'quarter']}

Advising Bot Response:

Josh Archer teaches Data Structures.


### Try it!

1. Try adding a tool that allows the LLM to see all of the names of classes students are taking and what their grades are.

2. If you have time, try adding a tool that allows the LLM to UPDATE or INSERT records based on user requests.

### Agents

In [74]:
import json

# Tool definitions
tools = [
    {
        "type": "function",
        "function": {
            "name": "sql_query",
            "description": "Execute a SQL query on the database and return the results.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The SQL query to execute."
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "clarify_user",
            "description": "Ask the user a clarifying question if necessary.",
            "parameters": {
                "type": "object",
                "properties": {
                    "message": {
                        "type": "string",
                        "description": "The question to ask the user."
                    }
                },
                "required": ["message"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "done",
            "description": "Indicate that the task is complete and exit the loop.",
            "parameters": {
                "type": "object",
                "properties": {
                    "message": {
                        "type": "string",
                        "description": "The final response to the user."
                    }
                },
                "required": ["message"]
            }
        }
    }
]

# Tool execution functions
def sql_query_tool(params):
    try:
        query = params["query"]
        cursor = conn.cursor()
        cursor.execute(query)
        results = cursor.fetchall()
        column_names = [description[0] for description in cursor.description]
        return {"success": True, "results": results, "columns": column_names}
    except Exception as e:
        return {"success": False, "error": str(e)}

def clarify_user_tool(params):
    question = params["message"]
    print("Clarifying Question:", question)
    return input("User Response: ")

def done_tool(params):
    done_message = params["message"]
    print("Done:", done_message)
    return {"success": True, "message": done_message}

# Execute selected tool
def execute_tool(tool_name, params={}):
    # Map tool functions
    tool_functions = {
        "sql_query": sql_query_tool,
        "clarify_user": clarify_user_tool,
        "done": done_tool
    }

    if tool_name in tool_functions:
        return tool_functions[tool_name](params)
    else:
        raise ValueError(f"Unknown tool: {tool_name}")


# React agent workflow
def react_agent_workflow(user_query, max_iterations=3):

        # Provide schema info to the LLM
    table_info = """
        \nThe database has the following tables:
        \ncourses (id, name, description, instructor_id, quarter)
        \ninstructors (id, name, email, department, office)
        \nstudents (id, name, email, advisor_id)
        \ngrades (id, student_id, course_id, grade)
    """

    # Initialize LLM messages

    messages = add_system_message([], f"""
        You are a database assistant for a college. Use tools to interact with the database or the user to fulfill the query. 
        Explain your thought process step by step and plan what you will need to do before calling tools.
        {table_info}
    """)
    
    messages = add_user_message(messages, user_query)
    
    iteration = 0

    while True and iteration < max_iterations:
        iteration += 1

        # Call the LLM to decide what to do
        response = generate_response(
            messages=messages,
            tools=tools,
            model="llama-3.1-70b-versatile",
            max_tokens=200,
            temperature=.2
        )

        # Print the LLM response
        print(colored("LLM Response:", "cyan"), colored(response.content, "blue"))
        print(colored("Tool Calls:", "cyan"), colored(response.tool_calls, "blue"))
        print(colored("DEBUG:", "red"), colored(response, "red"))

        # Extract the tool call from the response
        if response.tool_calls:
            tool_name = response.tool_calls[0].function.name
            # parse the arguments string to json
            tool_params = json.loads(response.tool_calls[0].function.arguments)

            # Execute the selected tool
            tool_result = execute_tool(tool_name, tool_params)
            print(colored(f"Tool Result ({tool_name}):", "green"), colored(tool_result, "blue"))

            if tool_name == "done":
                # Exit the loop when 'done' is called
                return tool_result["message"]

            # Add tool results or errors to context
            if tool_name == "sql_query" and tool_result["success"]:
                # Add successful query results
                context = "Here are the query results:\n"
                for row in tool_result["results"]:
                    row_details = [f"{col}: {val}" for col, val in zip(tool_result["columns"], row)]
                    context += ", ".join(row_details) + "\n"
                messages = add_ai_message(messages, context)
            elif tool_name == "sql_query" and not tool_result["success"]:
                # Add query error to context
                messages = add_ai_message(messages, f"Query failed with error: {tool_result['error']}")
            elif tool_name == "clarify_user":
                # Add user's clarification response to context
                clarification = tool_result
                messages = add_user_message(messages, clarification)
        else:
            # Add the LLM's direct response if no tool calls are made
            messages = add_ai_message(messages, response.content)
        print(colored("Messages:", "cyan"), colored(messages, "blue"))

print("\nReact Agent Workflow:\n")
user_query = input("Input your question: ")  # Example: "Show me all students."
response = react_agent_workflow(user_query)
print("\nReact Agent Response:\n")


React Agent Workflow:

LLM Response: None
Tool Calls: [ChoiceMessageToolCall(id='call_wv32', function=ChoiceMessageToolCallFunction(arguments='{"query": "SELECT * FROM
courses;"}', name='sql_query'), type='function')]
DEBUG: ChoiceMessage(content=None, role='assistant', tool_calls=[ChoiceMessageToolCall(id='call_wv32',
function=ChoiceMessageToolCallFunction(arguments='{"query": "SELECT * FROM courses;"}', name='sql_query'), type='function')])
Tool Result (sql_query): {'success': True, 'results': [(1, 'Introduction to AI', 'An introduction to artificial intelligence concepts
and techniques.', 5, 'Fall'), (2, 'Machine Learning', 'Covers supervised and unsupervised learning algorithms.', 2, 'Winter'), (3, 'Data Structures',
'Introduction to data structures like lists, trees, and graphs.', 3, 'Spring'), (4, 'Operating Systems', 'Covers the design and implementation of
operating systems.', 4, 'Fall'), (5, 'Web Development', 'Learn to build full-stack web applications.', 1, 'Winter')], 'col